In [1]:
import findspark

In [2]:
findspark.init('/home/pushya/spark-2.1.0-bin-hadoop2.7')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('mytree').getOrCreate()

In [6]:
data=spark.read.csv('movielens_ratings.csv',inferSchema='true',header='true')

In [7]:
from pyspark.ml.recommendation import ALS

In [8]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [10]:
train_data,test_data=data.randomSplit([0.8,0.2])

In [11]:
als = ALS(maxIter=5,
    regParam=0.01,userCol='userId',
    itemCol='movieId',
    seed=None,
    ratingCol='rating')

In [23]:
model = als.fit(train_data).transform(test_data)

In [13]:
model.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    19| -0.6584156|
|     31|   1.0|    24| -0.9512068|
|     31|   3.0|    14|  1.8634661|
|     85|   1.0|    28| 0.55852723|
|     85|   3.0|     1|   2.333302|
|     85|   1.0|     5|  2.7867846|
|     85|   1.0|    23|   3.611712|
|     85|   1.0|    25|   3.129239|
|     85|   1.0|    29|   2.552377|
|     65|   1.0|    28|   3.696071|
|     65|   1.0|    22| -0.5060822|
|     65|   2.0|     3|  -0.492885|
|     65|   2.0|     5|  1.8607591|
|     65|   1.0|     2|-0.87205744|
|     53|   1.0|     6|  1.4849343|
|     53|   1.0|     9|  1.2319232|
|     53|   1.0|    23|  2.2219343|
|     53|   1.0|     7|  2.0082784|
|     53|   1.0|    25|  4.3257427|
|     53|   5.0|    21|   2.727947|
+-------+------+------+-----------+
only showing top 20 rows



In [14]:
from pyspark.ml.evaluation import RegressionEvaluator

In [15]:
evaluator = RegressionEvaluator(
    predictionCol='prediction',
    labelCol='rating',
    metricName='rmse',

)

In [16]:
rmse= evaluator.evaluate(model)

In [17]:
print(rmse)
print('RMSE')

1.9150819442622975
RMSE


In [19]:
single_user = model.filter(model['userId']==11).select(['movieId','userId'])

In [20]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     72|    11|
|     39|    11|
|     90|    11|
|     66|    11|
|      0|    11|
|     18|    11|
|     36|    11|
+-------+------+



In [24]:
recommendations = model.transform(single_user)

In [25]:
recommendations.show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     72|    11|0.40415865|
|     39|    11|-0.5681932|
|     90|    11|  3.694215|
|     66|    11| -1.378015|
|      0|    11|-1.2790904|
|     18|    11| 1.1408473|
|     36|    11|0.29437095|
+-------+------+----------+

